Autor: Matyáš Sládek <br>
Rok: 2020 <br>

Tento soubor obsahuje funkce pro výpis výsledků selekce atributů, optimalizace parametrů či klasifikace.

Tato buňka vypíše informace o optimalizaci parametrů. <br>
V proměnné <code>datasets</code> je možné vybrat, pro které datové sady mají být informace vypasány. <br>
V proměnné <code>feature_extraction_libraries</code> je možné vybrat, pro atributy které z extrakčních knihoven mají být informace vypasány. <br>
V proměnné <code>feature_sets</code> je možné vybrat, pro které sady atributů mají být informace vypasány. <br>
V proměnné <code>classifiers</code> je možné vybrat, pro které klasifikátory mají být informace vypasány. <br>
V proměnné <code>validation_types</code> je možné vybrat, pro které typy validací (u optimalizace) mají být informace vypasány. <br>
Nastavením proměnné <code>show_all_trials</code> na hodnotu True je možné vysat informace o každé iteraci optimalizace.

In [ ]:
import joblib
from datetime import timedelta
import optuna
import copy

def get_duration(elem):
    return(elem.datetime_complete - elem.datetime_start)

datasets = [
#     'EBD',
#     'FMA',
    'GTZAN'
]

feature_extraction_libraries = [
#     'librosa',
    'essentia'
]

feature_sets = [
    'all',
#     'opt_feature_set_FS_VS',
#     'opt_feature_set_BE_VS'
]

classifiers = [
        # sklearn classifiers
#         'LogisticRegression',
        'KNeighborsClassifier',
#         'MLPClassifier',
#         'DecisionTreeClassifier',
#         'SVC_linear',
#         'SVC_rbf',
        
        # sklearn ensemble classifiers
#         'RandomForestClassifier',
        
        # other classifiers
#         'XGBClassifier'
]

validation_types = [
    'VS',
    'CV'
]

show_all_trials = False

for dataset in datasets:
    print('#'*100)
    print('Results for dataset \033[1m{}\033[0m:'.format(dataset))
    
    for library in feature_extraction_libraries:
        print('*'*100)
        print('Results for library \033[1m{}\033[0m:'.format(library))
        for feature_set in feature_sets:
            print('-'*100)
            print('Results for feature_set \033[1m{}\033[0m:'.format(feature_set))
            for classifier in classifiers:
                for validation_type in validation_types:
                    study = joblib.load('../metadata/optuna_studies/{}_{}_{}_{}_{}.pkl'.format(dataset, library, feature_set, classifier, validation_type)) 
                    print('')

                    if show_all_trials:
                        for trial in study.trials:
                            if trial.state == optuna.trial.TrialState.COMPLETE:
                                print('{} {} {} {} {}'.format(trial.number, trial.state, trial.value, str(trial.datetime_complete - trial.datetime_start).split('.')[0], trial.user_attrs['params']))
                            else:
                                print('{} {}'.format(trial.number, trial.state))

                    print('\nOptimisation info about classifier \033[1m{}\033[0m with validation type \033[1m{}\033[0m:'.format(classifier, validation_type))
                    print('Total optimisation runtime: \033[1m{}\033[0m\n'.format(str(study.trials[len(study.trials)-1].datetime_complete - study.trials[0].datetime_start).split('.')[0]))
                    print('Best trial:\n\tNumber:   \033[1m{}\033[0m\n\tScore:    \033[1m{}\033[0m\n\tRuntime:  \033[1m{}\033[0m\n\tParams:   \033[1m{}\033[0m'.format(study.best_trial.number, study.best_trial.value, str(study.best_trial.datetime_complete - study.best_trial.datetime_start).split('.')[0], study.best_trial.user_attrs['params']))

Tato buňka vypíše informace o selekci atributů. <br>
V proměnné <code>datasets</code> je možné vybrat, pro které datové sady mají být informace vypasány. <br>
V proměnné <code>feature_extraction_libraries</code> je možné vybrat, pro atributy které z extrakčních knihoven mají být informace vypasány. <br>
V proměnné <code>feature_sets</code> je možné vybrat, pro které sady atributů mají být informace vypasány. <br>
V proměnné <code>classifiers</code> je možné vybrat, pro které klasifikátory mají být informace vypasány. <br>

In [ ]:
import pandas as pd
import numpy as np
import sys
import json

try:
    with open('../metadata/misc/optimised_feature_sets.json') as f:
        optimised_feature_sets = json.load(f)   
except Exception as e:
    print('Failed to read file: "../metadata/misc/optimised_feature_sets.json"!', file=sys.stderr)
    print('Error: {}'.format(repr(e)), file=sys.stderr)
    
datasets = [
    'EBD',
#     'FMA',
#     'GTZAN'
]

feature_extraction_libraries = [
#     'librosa',
    'essentia'
]

feature_sets = [
    'opt_feature_set_FS_VS',
    'opt_feature_set_BE_VS'
]

classifiers = [
        # sklearn classifiers
#         'LogisticRegression',
        'KNeighborsClassifier',
#         'MLPClassifier',
#         'DecisionTreeClassifier',
#         'SVC_linear',
#         'SVC_rbf',
        
        # sklearn ensemble classifiers
#         'RandomForestClassifier',
        
        # other classifiers
#         'XGBClassifier'
]

for dataset in datasets:
    print('#'*100)
    print('Results for dataset \033[1m{}\033[0m:'.format(dataset))
    
    for library in feature_extraction_libraries:
        print('*'*100)
        print('Results for library \033[1m{}\033[0m:'.format(library))
        for feature_set_name in feature_sets:
            print('-'*100)
            print('Results for feature_set \033[1m{}\033[0m:'.format(feature_set_name))
            for classifier in classifiers:
                feature_set = optimised_feature_sets[dataset][library][classifier][feature_set_name]
                print('Feature subset info for classifier \033[1m{}\033[0m:'.format(classifier))
                print('Features selected: \033[1m{}\033[0m'.format(len(feature_set)))
                print('Features: \033[1m{}\033[0m'.format(feature_set))

Tato buňka vypíše informace o dosaženém skóre a době trénování a klasifikace pro vybrané klasifikační algoritmy sady atributů. <br>
V proměnné <code>datasets</code> je možné vybrat, pro které datové sady mají být informace vypasány. <br>
V proměnné <code>feature_extraction_libraries</code> je možné vybrat, pro atributy které z extrakčních knihoven mají být informace vypasány. <br>
V proměnné <code>feature_sets</code> je možné vybrat, pro které sady atributů mají být informace vypasány. <br>
V proměnné <code>control_sets</code> je možné vybrat, pro které testovací/validační sady mají být informace vypasány. <br>
Nastavením proměnné <code>load_f1_score</code> na hodnotu True je možné načíst hodnoty F1 skóre, při hodnotě False se vypíše klasické skóre (počet správně / počet celkem) <br>

In [ ]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

if __name__ == "__main__":
       
    def highlight_max(s):
        is_max = s == s.max()
        return ['background-color: yellow' if v else '' for v in is_max]
        
    datasets = [
        'EBD',
        'FMA',
        'GTZAN'
    ]
    
    feature_extraction_libraries = [
#         'librosa',
        'essentia'
    ]
    
    feature_sets = [
        'all',
        'opt_feature_set_FS_VS',
        'opt_feature_set_BE_VS'
    ]
    
    control_sets = [
#         'validation_set',
        'test_set'
    ]
    
    load_f1_score = False

    scs = pd.DataFrame()
    rts = pd.DataFrame()
    
    for dataset in datasets:
        for library in feature_extraction_libraries:
            for control_set in control_sets:
                
                scores = pd.read_csv("../metadata/scores/scores_{}_{}_{}_{}.csv".format(dataset, library, control_set, 'F1_score' if load_f1_score else 'accuracy_score'), index_col=0, header=[0])
                scs = scs.append(scores)
                runtimes = pd.read_csv("../metadata/runtimes/runtimes_{}_{}_{}.csv".format(dataset, library, control_set), index_col=0, header=[0])
                rts = rts.append(runtimes)

#     cols = [c for c in scs.columns if 'default' not in c]
#     cols = [c for c in cols if 'VS' not in c]
#     scs = scs[cols]
#     rts = rts[cols]
     
    index = scs.index.unique()
    index = pd.MultiIndex.from_product([['EBD', 'FMA', 'GTZAN'], list(index)])
    scs = scs.reset_index(drop=True)
    scs = pd.DataFrame(data=scs.values, index=index, columns=scs.columns.str.replace('_default',''))
    indices_to_drop = [x for x in ['all', 'opt_feature_set_FS_VS', 'opt_feature_set_BE_VS'] if x not in feature_sets]
    if len(indices_to_drop) != 0:
        scs = scs.drop(indices_to_drop, level=1)
    scs.to_csv('../../bp/classification_scores.csv')  
    scs = scs.style.apply(highlight_max, axis=1)
    ipd.display(scs.format("{:.2%}"))

    index = rts.index.unique()
    index = pd.MultiIndex.from_product([['EBD', 'FMA', 'GTZAN'], list(index)])
    rts = rts.reset_index(drop=True)
    rts = pd.DataFrame(data=rts.values, index=index, columns=rts.columns.str.replace('_default',''))
    if len(indices_to_drop) != 0:
        rts = rts.drop(indices_to_drop, level=1)
    rts.to_csv('../../bp/classification_runtimes.csv')  
    ipd.display(rts)